In [19]:
import pandas as pd
from random import randrange
import math
import difflib

In [20]:
data_trees = pd.read_excel('alberi_trento.xls')
species_list_xlsx = pd.read_excel('Eco_Species_List.xlsx')

In [21]:
data_trees.columns

Index(['Unnamed: 0', 'id_zril', 'code_istat', 'zona', 'area', 'obj_id', 'tp',
       'ts', 'codice', 'data_ini', 'data_fine', 'data_agg', 'modif_da', 'note',
       'foto', 'pt', 'genere', 'specie', 'varieta', 'h_m', 'diam_tronc',
       'diam_chiom', 'data_abbat', 'stato', 'longitude', 'latitude'],
      dtype='object')

In [22]:
# def removeWhitespace(name):
#     while name[-1:]==' ':
#         name = name[:-1]
#     while name[0:1]==' ':
#         name = name[1:]
#     return name

# species_list_xlsx['Scientific Name'] = species_list_xlsx.apply(lambda x : specFixer(x['Scientific Name']), axis=1)
species_list = species_list_xlsx['Scientific Name'].values
species_list

array(['Cycadopsida', 'Filicopsida', 'Ginkgoopsida', ..., 'Zygia',
       'Zygia longifolia', 'Zygophyllum'], dtype=object)

In [23]:
print('Platanus x acerifolia' in species_list)

True


In [24]:
#crea colonna della specie con nome scientifico, data da genere + specie, e pulisce le entries vuote. Droppa anche la colonna varieta' che non ci e' utile

data_trees['Species'] = data_trees['genere'] + ' ' + data_trees['specie']
data_trees = data_trees.drop('genere',1)
data_trees = data_trees.drop('specie',1)
data_trees = data_trees.drop('varieta',1)
data_trees = data_trees.dropna(subset=['Species'])

C:\Users\Giacomo\AppData\Local\Temp\ipykernel_23708\1397991911.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_trees = data_trees.drop('genere',1)
C:\Users\Giacomo\AppData\Local\Temp\ipykernel_23708\1397991911.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_trees = data_trees.drop('specie',1)
C:\Users\Giacomo\AppData\Local\Temp\ipykernel_23708\1397991911.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_trees = data_trees.drop('varieta',1)


In [25]:
#print di ogni stato delle piante

data_trees['stato'].unique()

array(['Pianta viva', 'Ceppaia', 'Pianta abbattuta', 'Pianta non gestita',
       'Viva', 'Pianta morta', 'Posto libero'], dtype=object)

In [26]:
#drop di alberi con altezza maggiore di 100m e drop di entries con piante morte ecc.

data_trees = data_trees.drop(data_trees[data_trees['h_m'] >=100].index)
data_trees = data_trees.drop(data_trees[data_trees['diam_tronc'] >=500].index)
data_trees = data_trees.drop(data_trees[(data_trees['stato'] != 'Pianta viva') & (data_trees['stato'] != 'Viva') & (data_trees['stato'] != 'Pianta non gestita')].index)

data_trees['stato'].unique()

array(['Pianta viva', 'Pianta non gestita', 'Viva'], dtype=object)

In [27]:
#drop di entries con piante morte ecc. tramite il campo 'note'

data_trees = data_trees.drop(data_trees[(data_trees['note'] == 'ALB MORTA') | (data_trees['note'] == 'Pianta segnata come eseguita ma in realtà non è stata fatta') | (data_trees['note'] == 'MORTA ALBERATE') |(data_trees['note'] == 'MORTA')].index)

In [28]:
#Pulisce le specie dai suffissi spp. o simili, e pulisce gli spazi bianchi a inizio nome + fine nome.
#Cerca di sostituire le specie scritte male / sbagliate con quelle giuste facendo un confronto con le piu' simili in una lista di specie presa direttamente dal sito di ITreeTools.

open("Substitute species.txt", "w").close()

def specFixer(name):
    if(name[-4:] == 'spp.'):
        name = name[:-4]
    elif(name[-3:] == 'spp' or name[-3:] == 'sp.'):
        name = name[:-3]
    elif(name[-2:] == 'sp'):
        name = name[:-2]
    while name[-1:]==' ':
        name = name[:-1]
    while name[0:1]==' ':
        name = name[1:]
    if(name not in species_list):
        f = open("Substitute species.txt", "a")
        best_match = difflib.get_close_matches(name,species_list,1)
        #print(name + ' -- ' + str(best_match))
        if(best_match != []):
            best_match = best_match[0]
            score = difflib.SequenceMatcher(None, name, best_match).ratio()
            f.write(name + "  -->  " + best_match + "\nScore: " +  str(score) + '\n\n')
            name = best_match
        else:
            f.write(name + "  -->  None \n\n")
        f.close()
    return name

data_trees['Species'] = data_trees.apply(lambda x : specFixer(x['Species']), axis=1)


In [29]:
#media altezza alberi nel dataset

data_trees['h_m'].describe()

count    11119.000000
mean         7.940939
std          4.966198
min          0.000000
25%          4.500000
50%          6.000000
75%         10.000000
max         60.000000
Name: h_m, dtype: float64

In [30]:
#media spessore tronco alberi nel dataset

data_trees['diam_tronc'].describe()

count    11028.000000
mean        28.037384
std         34.273040
min          0.000000
25%         12.000000
50%         20.000000
75%         35.000000
max       2515.000000
Name: diam_tronc, dtype: float64

In [31]:
#media diametro chioma alberi nel dataset

data_trees['diam_chiom'].describe()

count    11000.000000
mean         5.255227
std          3.542011
min          0.000000
25%          3.000000
50%          4.000000
75%          7.000000
max         26.000000
Name: diam_chiom, dtype: float64

In [32]:
#prende il nome di ogni specie, fa le stime sui dati di SOLO quella specie e applica i valori di MEDIA ad entries con lo stesso nome specie ma prive di dati utili. Nel caso non ci fossero abbastanza dati, inseriesce default MEDIA altezza ecc. preso da intero dataset.
#Log dei dati utili per ogni specie sul file "analyzeDataperSpecies.txt".
#Percentuale mancante chioma presa randomicamente da 1% a 35%
#Base della chioma settata a 70% altezza totale albero.
#valori fillati : altezza albero, diametro chioma, spessore tronco, percentuale mancante chioma e base della chioma.

specieNames = data_trees.Species.unique()

open("analyzeDataperSpecies.txt", "w").close()

rand_crown = ['1% - 5%','5% - 10%','10% - 15%', '15% - 20%', '20% - 25%', '25% - 30%', '30% - 35%']

zeros = [float(0),float(0.0)]

def dataFixer(item,heightMean,widthMean,crownWidthMean,species):
    if item['Species'] == species:
        if math.isnan(item['h_m']) or (item['h_m'] in zeros):
            item['h_m'] = heightMean
        if math.isnan(item['diam_tronc']) or (item['diam_tronc'] in zeros):
            item['diam_tronc'] = widthMean
        if math.isnan(item['diam_chiom']) or (item['diam_chiom'] in zeros):
            item['diam_chiom'] = crownWidthMean
        item['crown_base_height'] = float(item['h_m']) * 0.7
        count = randrange(0, 7)
        item['percent_crown_missing'] = rand_crown[count]
    #print(item.h_m)
    return item

for x in specieNames:
    contained_values_by_species = pd.DataFrame(data_trees[data_trees['Species'].str.contains(x)])
    heightMean = contained_values_by_species['h_m'].mean()
    widthMean = contained_values_by_species['diam_tronc'].mean()
    crownWidthMean = contained_values_by_species['diam_chiom'].mean()
    if math.isnan(heightMean) or (heightMean in zeros):
        heightMean = 7.95
    if math.isnan(widthMean) or (widthMean in zeros):
        widthMean = 28.23
    if math.isnan(crownWidthMean) or (crownWidthMean in zeros):
        crownWidthMean = 5.26
    f = open("analyzeDataperSpecies.txt", "a")
    f.write("\nAnalyzed species : " + x + '\n')
    f.write("Occasions : " + str(len(contained_values_by_species.index)) + '\n')
    f.write('Values : \n' + str(contained_values_by_species['h_m'].describe()) + '\n\n' + str(contained_values_by_species['diam_tronc'].describe()) + '\n\n' + str(contained_values_by_species['diam_chiom'].describe()) + '\n\n')
    f.write('Values to use :\nheightMean=' + str(heightMean) + '\nwidthMean=' + str(widthMean) + '\ncrownWidthMean=' + str(crownWidthMean) + '\n\n\n')
    f.close()
    data_trees = data_trees.apply(lambda y : dataFixer(y,heightMean,widthMean,crownWidthMean,x), axis=1)



In [34]:
#creazione di una colonna ulteriore per la chioma in modo che itreetools non dia errori (clonata dai dati gia' esistenti)
#Rinominazione colonna iniziale.

data_trees['crown_N_S'] = data_trees['diam_chiom']
data_trees['crown_total_height'] = data_trees['h_m']
data_trees.rename(columns={'diam_chiom':'crown_E_O'},inplace=True)


In [36]:
#salva dataFrame in un file excel 'xlsx'.

data_trees.to_excel('modifiedDataset.xlsx')